In [17]:
using LinearAlgebra
using JuMP
using HiGHS
using SDDP
using Statistics
using Plots
using Random
using Printf

In [2]:
const conv = (30*24*60*60)/1e6

2.592

In [3]:
mutable struct Data
    T::Int
    S::Int
    N::Int
    cost::Vector{Float64}
    rho::Float64
    generation_max::Vector{Float64}
    volume_initial::Float64
    volume_max::Float64
    turbine_max::Float64
    demand::Vector{Float64}
    scenarios::Vector{Vector{Float64}}
end

data = Data(3, #T
           2, #S
           5, #N
           [10,20,40,80,500], #cost
           0.96, #rho
           [100,150,200,250,1000], #generation_max
           2050, #volume_initial
           4100, #volume_max
           1500, #turbine_max
           [1000,1000,1000], #demand
           [[150],[450,300],[771,213]] #scenarios
           );

function unzip(data::Data)
    T = data.T
    S = data.S
    N = data.N
    cost = data.cost
    rho = data.rho
    generation_max = data.generation_max
    volume_initial = data.volume_initial
    volume_max = data.volume_max
    turbine_max = data.turbine_max
    demand = data.demand
    scenarios = data.scenarios
    
    return T, S, N, cost, rho, generation_max, volume_initial, volume_max, turbine_max, demand, scenarios
end

unzip (generic function with 1 method)

In [4]:
function tree(node,S)
    return [(node...,s) for s in 1:S]
end

function create_paths(T, S)
    temp = Int.(Tuple([1, (zeros(T-1).+S)...]))
    paths = []
    foreach(path->push!(paths,Tuple(path)),CartesianIndices(temp))
    return paths
end

create_paths (generic function with 1 method)

## 1) Implement and solve the deterministic equivalent for the hydrothermal scheduling problem in Section 6.

In [5]:
function deterministic_equivalent(data::Data)
    
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    
    T, S, N, cost, rho, generation_max, volume_initial, volume_max, turbine_max, demand, scenarios = unzip(data)
    
    idxs = create_paths.(1:T,S)
    idxs_vol = create_paths.(1:T+1,S)
    
    volume = model[:volume] = []
    omega = model[:omega] = []
    turbined = model[:turbined] = []
    thermal_generation = model[:thermal_generation] = []
    spilled = model[:spilled] = []
    
    @variable(model, objective_stage[t in 1:T], base_name="obj", lower_bound = 0)
    for (t, idx) in enumerate(idxs_vol)
        push!(volume, @variable(model, [i in idx], base_name="volume_$t", lower_bound = 0, upper_bound =  data.volume_max))
    end
    for (t, idx) in enumerate(idxs)
        push!(omega, @variable(model, [idx], base_name="omega_$t", lower_bound = 0))
        push!(turbined, @variable(model, [idx], base_name="turbined_$t", lower_bound = 0))
        push!(thermal_generation, @variable(model, [idx, 1:N], base_name="thermal_generation_$t", lower_bound = 0))
        push!(spilled, @variable(model, [idx], base_name="spilled_$t", lower_bound = 0))
    end

    @constraint(model, volume[1][(1,)] == data.volume_initial)
    for t = 1:T
        for scen in idxs[t]
            s = scen[t]
            inflow = scenarios[t][s]
            obj = sum(cost[n]*thermal_generation[t][scen,n] for n in 1:N)
            @constraint(model, omega[t][scen] == obj)
            @constraint(model, turbined[t][scen]*rho + sum(thermal_generation[t][scen,n] for n in 1:N) == demand[t])
            @constraint(model, turbined[t][scen] <= data.turbine_max)
            @constraint(model, [n in 1:N], thermal_generation[t][scen,n] <= data.generation_max[n])
            for node in tree(scen,S)
                @constraint(model, volume[t+1][node] == volume[t][scen] + conv*(inflow - turbined[t][scen] - spilled[t][scen]))
            end
        end
        @constraint(model, objective_stage[t] == 1/length(idxs[t]).*sum([omega[t]...]))
    end
    @objective(model, Min, sum(objective_stage));
    return model
end

deterministic_equivalent (generic function with 1 method)

In [6]:
model = deterministic_equivalent(data)
optimize!(model)
objective_value(model)

38008.69629629629

## 2) Implement the nested Benders decomposition also detailed in Section 6.

In [7]:
struct Cut
    pi::Union{Float64,Array{Float64}}
    Q::Float64
    x::Union{Float64,Array{Float64}}
end

struct FCF
    cuts::Array{Cut}
    stage::Int64
end

In [34]:
function add_cuts!(model::JuMP.Model, FCF::FCF)
    x = model[:volume_out]
    model[:omega_t] = omega_t = @variable(model, base_name = "omega_$(FCF.stage)")
    for cut in FCF.cuts
        @constraint(model, omega_t >= sum(cut.pi .* (x .- cut.x)) + cut.Q)
    end
    objective = objective_function(model)
    objective += omega_t
    set_objective_function(model, objective)
end

add_cuts! (generic function with 1 method)

In [35]:
function subproblem(data::Data, volume_in::Float64, FCF::FCF, inflow::Float64)
    
    t = FCF.stage
    T, S, N, cost, rho, generation_max, volume_initial, volume_max, turbine_max, demand, scenarios = unzip(data)
    
    model = Model()
    set_silent(model)

    @variable(model, 0 <= thermal_generation[n=1:N] <= generation_max[n])
    @variable(model, 0  <= turbined <= turbine_max)
    @variable(model, spilled >= 0)
    @variable(model, 0 <= volume_out <= volume_max)
    @variable(model, dual_fisher)
    
    @constraint(model, fisher, dual_fisher == volume_in)
    @constraint(model, volume_out == dual_fisher + conv*(inflow - turbined - spilled))
    @constraint(model, sum(thermal_generation) + turbined*rho == demand[t])
        
    @objective(model,Min, sum(cost[n] * thermal_generation[n] for n = 1:N))
    add_cuts!(model, FCF)
    return model
end

subproblem (generic function with 1 method)

In [36]:
Ω = [
    [150.0],
    [[450.0, 300.0]],
    [[771.0, 213.0],[771.0, 213.0]]
]

3-element Vector{Vector}:
 [150.0]
 [[450.0, 300.0]]
 [[771.0, 213.0], [771.0, 213.0]]

In [77]:
function NBD(data, Ω; maxiter = maxiter, timelimit = Inf)
    T, S, N, cost, rho, generation_max, volume_initial, volume_max, turbine_max, demand, scenarios = unzip(data)
    iter = 1
    FCFs = FCF[FCF(Cut[Cut(zeros(1),0.0,zeros(1))],t) for t = 1:length(scenarios)]
    println("Iter |       LB         |       UB         |  Time")
    println()
    t_1 = time_ns()
    time = 0.0
    K = length.(scenarios)
    LBs = []
    UBs = []
    times = []
    idxs = create_paths.(1:T,K)
    while iter <= maxiter && time <= timelimit
        xs = [[] for t = 1:T]
        pi_xis = [[] for t = 1:T]
        Q_xis = [[] for t = 1:T]
        uks = [[] for t = 1:T]
        for t = 1:T
            for (k,St) in enumerate(Ω[t])
                pi_o = []
                Q_o = []
                uk_o = []

                for inflow in St
                    if t == 1
                        model = subproblem(data,volume_initial,FCFs[t],inflow)
                    else
                        model = subproblem(data,xs[t-1][k],FCFs[t],inflow)
                    end
                    set_optimizer(model, HiGHS.Optimizer)
                    set_silent(model)
                    optimize!(model)
                    value(model[:volume_out])
                    push!(xs[t],value(model[:volume_out]))
                    pi_xi = dual(constraint_by_name(model, "fisher"))
                    fcf = value(variable_by_name(model, "omega_$(t)"))
                    obj_val = objective_value(model)
                    stage_obj_val = objective_value(model) - fcf
                    push!(pi_o,pi_xi)
                    push!(Q_o,obj_val)
                    push!(uk_o,stage_obj_val)
                end
                push!(pi_xis[t],mean(pi_o))
                push!(Q_xis[t],mean(Q_o))
                push!(uks[t],mean(uk_o))
            end
        end
        UB = sum(mean.(uks))

         # Backward Step
        for t = T:-1:2
            for (k,St) in enumerate(Ω[t])
                cut = Cut(pi_xis[t][k],Q_xis[t][k],xs[t-1][k])
                push!(FCFs[t-1].cuts,cut)
            end
        end

        ω_1 = scenarios[1][1]
        model = subproblem(data,volume_initial,FCFs[1],ω_1)
        set_optimizer(model, HiGHS.Optimizer)
        set_silent(model)
        optimize!(model)
        LB = objective_value(model)

        # Get elapsed time
        t_2 = time_ns()
        time = (t_2-t_1)/1e9
        
        push!(LBs,LB)
        push!(times,time)

        @printf("%3d  |    %11.5e   |    %11.5e   | %5.3f\n", iter, LB, UB, time)

        iter = iter + 1
    end
    return FCFs,LBs,UBs,times
end

NBD (generic function with 1 method)

In [78]:
FCFs,LBs,UBs,times = NBD(data, Ω, maxiter = 10);

Iter |       LB         |       UB         |  Time

  1  |    2.98447e+03   |    7.27642e+04   | 0.004
  2  |    1.05809e+04   |    4.81288e+04   | 0.008
  3  |    3.78201e+04   |    3.88069e+04   | 0.012
  4  |    3.80087e+04   |    3.80087e+04   | 0.016
  5  |    3.80087e+04   |    3.80087e+04   | 0.020
  6  |    3.80087e+04   |    3.80087e+04   | 0.025
  7  |    3.80087e+04   |    3.80087e+04   | 0.029
  8  |    3.80087e+04   |    3.80087e+04   | 0.033
  9  |    3.80087e+04   |    3.80087e+04   | 0.037
 10  |    3.80087e+04   |    3.80087e+04   | 0.042


## 3) Implement the same model in the SDDP.jl package.

In [15]:
model = SDDP.LinearPolicyGraph(
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do subproblem, node
    
    T, S, N, cost, rho, generation_max, volume_initial, volume_max, turbine_max, demand, scenarios = unzip(data)
    
    model = Model()
    set_silent(model)
    demand = demand[node]
    
    @variable(subproblem, 0 <= volume <= volume_max, SDDP.State, initial_value = volume_initial)
    @variable(subproblem, 0 <= thermal_generation[n=1:N] <= generation_max[n])
    @variable(subproblem, 0 <= turbined <= turbine_max)
    @variable(subproblem, spilled >= 0)

    @variable(subproblem, inflow)
    scenarios = scenarios[node]
    K= length(scenarios)
    P = (1/K)*ones(K)
    SDDP.parameterize(subproblem, scenarios, P) do ω
        return JuMP.fix(inflow, ω)
    end

    @constraint(subproblem, volume.out == volume.in + conv*(inflow - turbined - spilled))
    @constraint(subproblem, sum(thermal_generation) + turbined*rho == demand)

    @stageobjective(subproblem, sum(cost[n] * thermal_generation[n] for n = 1:N))
end

SDDP.train(model; iteration_limit = 10)

------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 3
  State variables : 1
  Scenarios       : 4.00000e+00
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (11, 11)
    VariableRef in MOI.LessThan{Float64}    : (7, 8)
    VariableRef in MOI.GreaterThan{Float64} : (9, 9)
    AffExpr in MOI.EqualTo{Float64}         : (2, 2)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [1e+00, 3e+00]
  Non-zero Objective range  [1e+00, 5e+02]
  Non-zero Bounds range     [1e+02, 4e+03]
  Non-zero RHS range        [1e+03, 1e+03]
No problems detected

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        1    1.021674e+05   4.026180e+03   0.000000e+00      